In [127]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
searchInfo = "42+Match"
tempURL = "https://sailboatdata.com/sailboat?filter%5Bname%5D="+ searchInfo +"&paginate=25&units=imperial&sort=name"

headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
page = requests.get(tempURL, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

In [4]:
resultTable =  soup.select("div.table-responsive tbody td a")
resultLinks = []
for result in resultTable:
    link = result.attrs["href"]
    resultLinks.append(link+"?units=imperial")

print(resultLinks)

['https://sailboatdata.com/sailboat/bavaria-match-42?units=imperial']


####Page 2

In [5]:
# aimURL = resultLinks[0]
aimURL = "https://sailboatdata.com/sailboat/11-meter?units=imperial"
page = requests.get(aimURL, headers=headers)
soup = BeautifulSoup(page.content, "html.parser")

In [8]:
keys = []
values = []

keys.append("Make & Variant")
name = aimURL.split("?")[0].split("/")[-1]
values.append(name)

infoCursor = soup.select("div.sailboat-details div.row")
for i in range(len(infoCursor)):
    infoName = infoCursor[i].find(
        'div', attrs={"class": "sailboatdata-label"})
    if infoName == None:
        continue
    else:
        infoName = infoName.text.strip()
    infoValue = infoCursor[i].find(
        'div', attrs={"class": "sailboatdata-data"}).text.strip()
    keys.append(infoName)
    values.append(infoValue)
    if infoName == "Designer:":
        break
print(keys)
print(values)

['Make & Variant', 'Hull Type:', 'LOA:', 'Beam:', 'Draft (max):', 'Displacement:', 'S.A./Disp.:', 'Construction:', 'First Built:', 'Designer:']
['11-meter', 'Fin w/bulb & spade rudder', '33.80 ft', '8.20 ft', '5.90 ft', '3,527 lb', '31.16', 'FG w/core,Composite', '1990', 'Ron Holland & Rolf Gyhlenius']


In [105]:
from crawler import getLinkListFromURL, getSailboatInfoFromURL

def getSailboatFeatures(searchKey, year):
    if '&' in searchKey:
        searchKey = searchKey.replace('&',"%26")
    aimURL = "https://sailboatdata.com/sailboat?filter%5Bname%5D="+ searchKey +"&paginate=25&units=imperial&sort=name"
    answers = getLinkListFromURL(aimURL)
    # 有搜索结果
    if len(answers) > 0:
        # 有唯一的搜索结果
        if(len(answers) == 1):
            df_features = getSailboatInfoFromURL(answers[0])
        else:
            # 有多个搜索结果,比较制造年限选择
            for i in range(len(answers)):
                df_features = getSailboatInfoFromURL(answers[i])
                builtYear = int(df_features.loc[0,"First Built:"])
                if builtYear <= year:
                    break 
    # 无搜索结果
    else:
        return searchKey + " No Answer!" 
    
    return df_features

In [80]:

# x = getSailboatFeatures("C&C 121")
x = getSailboatFeatures("Lavezzi 40", 2005)
print(x)

      variant           Hull Type:      LOA:     Beam: Draft (max):  \
0  lavezzi-40  Catamaran Twin Keel  39.10 ft  21.40 ft      3.60 ft   

  Displacement: S.A./Disp.: Construction: First Built:  \
0     13,228 lb       27.82            FG         2002   

                           Builder:                             Designer:  
0  Fountaine Pajot Catamarans (FRA)  O. Flahault Design /Joubert - Nivelt  


In [140]:
originalFile = "../data/2023_MCM_Problem_Y_Boats.xlsx"

data_MonohulledSailboats = pd.read_excel(originalFile,sheet_name="Monohulled Sailboats ")
data_Catamarans = pd.read_excel(originalFile,sheet_name="Catamarans")

data_MonohulledSailboats["Class"] = "Monohulled"
data_Catamarans["Class"] = "Catamarans" 

dataSailboats = pd.concat([data_Catamarans, data_MonohulledSailboats]).reset_index(drop=True)

In [148]:
dataSailboats = pd.read_excel("./totalSailData.xlsx")
dataSailboats.reset_index(inplace=True)
df = dataSailboats.rename(columns={'index': 'ID'})
print(dataSailboats)

      index             Make     Variant  Length \n(ft) Geographic Region  \
0         0           Lagoon         380           38.0         Caribbean   
1         1           Lagoon         380           38.0         Caribbean   
2         2           Lagoon         380           38.0               USA   
3         3  Fountaine Pajot  Lavezzi 40           39.0         Caribbean   
4         4          Leopard          40           39.0         Caribbean   
...     ...              ...         ...            ...               ...   
3486   3486        RM Yachts    RM 12.70           40.0            Europe   
3487   3487        RM Yachts     RM 1270           42.0            Europe   
3488   3488        Southerly          48           48.0            Europe   
3489   3489         X-Yachts          X4           43.0            Europe   
3490   3490         X-Yachts        X4.6           46.0            Europe   

     Country/Region/State   Listing Price (USD)   Year       Class  
0     

In [139]:
sample = dataSailboats.iloc[116]
print(sample)
searchKey = dataSailboats.iloc[sampleNum]["Make"] + " " + str(dataSailboats.iloc[sampleNum]["Variant"])
searchYear = int(dataSailboats.iloc[sampleNum]["Year"])
searchResult = getSailboatFeatures(searchKey, searchYear)
print(searchResult)

Make                         Lagoon
Variant                      380 S2
Length \n(ft)                  38.0
Geographic Region            Europe
Country/Region/State          Italy
Listing Price (USD)          236926
Year                           2008
Class                    Catamarans
Name: 116, dtype: object
      variant           Hull Type:      LOA:     Beam: Draft (max):  \
0  lagoon-380  Catamaran Twin Keel  37.89 ft  21.42 ft      3.77 ft   

  Displacement: S.A./Disp.: Construction: First Built:        Builder:  \
0     16,005 lb       20.96            FG         2000  Jeanneau (FRA)   

                      Designer:  
0  Van Peteghem/Lauriot-Prevost  


In [122]:
test = data_MonohulledSailboats[:3]
for sampleNum in range(test.shape[0]):
    sample = test.iloc[sampleNum]
    searchKey = test.iloc[sampleNum]["Make"] + " " + test.iloc[sampleNum]["Variant"]
    searchYear = int(test.iloc[sampleNum]["Year"])
    searchResult = getSailboatFeatures(searchKey, searchYear)
    
    if sampleNum == 0:
        infoExtra = searchResult
    else:
        infoExtra = pd.concat([infoExtra, searchResult],ignore_index=True)
        
newTab = pd.concat([test, infoExtra], axis=1).reset_index(drop=True)

In [115]:
test

,Make,Variant,Length \n(ft),Geographic Region,Country/Region/State,Listing Price (USD),Year
0,Alubat,Ovni 395,41,Europe,France,267233,2005
1,Bavaria,38 Cruiser,38,Europe,Croatia,75178,2005
2,Bavaria,38 Cruiser,38,Europe,Croatia,66825,2005


In [116]:
infoExtra

,variant,Hull Type:,LOA:,Beam:,Draft (max):,Displacement:,S.A./Disp.:,Construction:,First Built:,Builder:,Designer:
0,alubat-58,Fin w/spade rudder,58.55 ft,16.81 ft,7.71 ft,"35,620 lb",26.82,Aluminum,2007,Alubat (FRA),P. Briand
1,bavaria-cruiser-38,Fin w/spade rudder,40.35 ft,12.67 ft,5.58 ft,"15,400 lb",19.48,FG,2000,Bavaria Yachts (GER),J & J Design
2,bavaria-cruiser-38,Fin w/spade rudder,40.35 ft,12.67 ft,5.58 ft,"15,400 lb",19.48,FG,2000,Bavaria Yachts (GER),J & J Design


In [121]:
pd.concat([test, infoExtra], axis=1).reset_index(drop=True)

,Make,Variant,Length \n(ft),Geographic Region,Country/Region/State,Listing Price (USD),Year,variant,Hull Type:,LOA:,Beam:,Draft (max):,Displacement:,S.A./Disp.:,Construction:,First Built:,Builder:,Designer:
0,Alubat,Ovni 395,41,Europe,France,267233,2005,alubat-58,Fin w/spade rudder,58.55 ft,16.81 ft,7.71 ft,"35,620 lb",26.82,Aluminum,2007,Alubat (FRA),P. Briand
1,Bavaria,38 Cruiser,38,Europe,Croatia,75178,2005,bavaria-cruiser-38,Fin w/spade rudder,40.35 ft,12.67 ft,5.58 ft,"15,400 lb",19.48,FG,2000,Bavaria Yachts (GER),J & J Design
2,Bavaria,38 Cruiser,38,Europe,Croatia,66825,2005,bavaria-cruiser-38,Fin w/spade rudder,40.35 ft,12.67 ft,5.58 ft,"15,400 lb",19.48,FG,2000,Bavaria Yachts (GER),J & J Design


In [109]:
test.join(infoExtra)

,Make,Variant,Length \n(ft),Geographic Region,Country/Region/State,Listing Price (USD),Year,variant,Hull Type:,LOA:,Beam:,Draft (max):,Displacement:,S.A./Disp.:,Construction:,First Built:,Builder:,Designer:
0,Alubat,Ovni 395,41,Europe,France,267233,2005,alubat-58,Fin w/spade rudder,58.55 ft,16.81 ft,7.71 ft,"35,620 lb",26.82,Aluminum,2007,Alubat (FRA),P. Briand
0,Alubat,Ovni 395,41,Europe,France,267233,2005,bavaria-cruiser-38,Fin w/spade rudder,40.35 ft,12.67 ft,5.58 ft,"15,400 lb",19.48,FG,2000,Bavaria Yachts (GER),J & J Design
0,Alubat,Ovni 395,41,Europe,France,267233,2005,bavaria-cruiser-38,Fin w/spade rudder,40.35 ft,12.67 ft,5.58 ft,"15,400 lb",19.48,FG,2000,Bavaria Yachts (GER),J & J Design
1,Bavaria,38 Cruiser,38,Europe,Croatia,75178,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bavaria,38 Cruiser,38,Europe,Croatia,66825,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
